<a href="https://colab.research.google.com/github/aadityane93/SortingLine-ml/blob/main/wavelets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
folder_id = '1ps_Bh93KQRJ4WbnilORmKd1BaWUXtmt1' # files

folder_list = drive.ListFile({'q': "'{}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false".format(folder_id)}).GetList()


for folder in folder_list:
    folder_title = folder['title']
    print(f"Processing folder: {folder_title}")
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder['id'])}).GetList()

    for index, file in enumerate(file_list):
        file_title = file['title']
        print(f"Downloading {file_title}")
        file.GetContentFile(file_title)

Processing folder: 400hz
Processing folder: 100hz
Processing folder: 800hz


In [23]:
import pandas as pd

In [27]:
train_target = pd.read_csv('100hz-train-target-50percent-10files.csv')
train_target.drop(columns=['Unnamed: 0'], inplace = True)
test_target = pd.read_csv('100hz-test-target-50percent-10files.csv')
test_target.drop(columns=['Unnamed: 0'], inplace = True)
train_feature = pd.read_csv('100hz-train-feature-50percent-10files.csv')
train_feature.drop(columns=['Unnamed: 0', 'Datum', 'Energie_In(Wh)','Energie_Out(Wh)', 'P_In(W)', 'P_Out(W)', 'color'], inplace = True)
test_feature = pd.read_csv('100hz-test-feature-50percent-10files.csv')
test_feature.drop(columns=['Unnamed: 0', 'Datum', 'Energie_In(Wh)','Energie_Out(Wh)', 'P_In(W)', 'P_Out(W)', 'color' ], inplace = True)

In [28]:
train_feature

,Uhrzeit,I_In,V_In,I_Out(A),V_Out(V),Temp(°C),fileno
0,2024-05-02 11:19:09.845032:845,0.25,11.52,0.23,9.00,37.13,0
1,2024-05-02 11:19:09.853083:853,0.29,11.65,0.25,9.00,36.63,0
2,2024-05-02 11:19:09.861520:861,0.39,11.65,0.33,9.03,37.13,0
3,2024-05-02 11:19:09.873394:873,0.27,11.61,0.41,9.03,37.13,0
4,2024-05-02 11:19:09.881917:881,0.25,11.74,0.25,9.03,37.13,0
...,...,...,...,...,...,...,...
15127,2024-05-02 12:51:02.302558:302,0.31,11.70,0.29,9.03,37.88,34
15128,2024-05-02 12:51:02.310972:310,0.25,11.61,0.31,9.06,37.88,34
15129,2024-05-02 12:51:02.323006:323,0.25,11.70,0.25,9.03,37.88,34
15130,2024-05-02 12:51:02.331326:331,0.31,11.70,0.27,9.00,37.88,34


!pip install --upgrade pip

In [25]:
!pip install pywt==1.3.0

import pywt
import numpy as np

ERROR: Could not find a version that satisfies the requirement pywt==1.3.0 (from versions: none)
ERROR: No matching distribution found for pywt==1.3.0


ModuleNotFoundError: No module named 'pywt'

In [37]:
wavelet_type = 'db4'
decomp_level = 3

In [38]:
x_train_sorted = train_feature.sort_values(by=['fileno', 'Uhrzeit'])

# Group by fileno (each group is a time series)
grouped = x_train_sorted.groupby('fileno')

In [32]:
wavelet_features = []

for fileno, group in grouped:
    signal = group['P_In(W)'].values  # Example: Use power input

    # Perform wavelet decomposition
    coeffs = pywt.wavedec(signal, wavelet_type, level=decomp_level)

    # Extract features from coefficients
    features = {}
    for i, coeff in enumerate(coeffs):
        if len(coeff) == 0:
            continue
        # features: energy, entropy, variance
        features[f'wavelet_energy_level_{i}'] = np.sum(coeff**2)
        features[f'wavelet_entropy_level_{i}'] = -np.sum(coeff**2 * np.log(coeff**2))
        features[f'wavelet_var_level_{i}'] = np.var(coeff)

    # Add fileno to map features back
    features['fileno'] = fileno
    wavelet_features.append(features)

# Convert to DataFrame
wavelet_df = pd.DataFrame(wavelet_features).set_index('fileno')

In [33]:
wavelet_df

,wavelet_energy_level_0,wavelet_entropy_level_0,wavelet_var_level_0,wavelet_energy_level_1,wavelet_entropy_level_1,wavelet_var_level_1,wavelet_energy_level_2,wavelet_entropy_level_2,wavelet_var_level_2,wavelet_energy_level_3,wavelet_entropy_level_3,wavelet_var_level_3
fileno,,,,,,,,,,,,
0,9018.000001,-42677.405249,3.094544,21.657628,-28.005913,0.251152,40.319574,-1.351644,0.244122,101.380909,-57.837948,0.314608
1,9045.940096,-42684.108872,2.874838,83.942425,-277.842590,0.964001,50.855689,-41.473230,0.305245,71.516924,35.743846,0.221398
2,9041.528154,-42567.653484,2.836852,31.125772,-46.302796,0.347618,43.902791,1.646225,0.262596,82.712330,2.983578,0.251181
3,8726.631321,-40883.127801,2.381344,26.702761,-30.282440,0.312611,35.310136,28.108521,0.212725,97.418971,-85.115223,0.302409
4,8808.437496,-41660.857578,2.971661,36.322672,-63.065383,0.437121,77.761000,-152.291239,0.486516,72.220531,11.900343,0.231025
10,10932.406174,-51070.947585,1.959451,32.867463,-36.396904,0.306824,68.033539,-60.256163,0.325311,102.322993,51.449027,0.249395
11,10672.665779,-50023.595524,2.117189,24.422555,-12.982619,0.235322,86.410784,-143.657900,0.434169,97.467763,34.568428,0.249253
12,10750.818390,-50444.802129,2.004128,63.291997,-152.205549,0.614414,39.594401,27.244911,0.198751,109.674594,-13.930339,0.279515
13,10962.867272,-51201.300585,1.815446,30.862112,-29.446913,0.286988,92.626186,-172.464246,0.443906,104.917127,42.823002,0.255731
